# Lab 1, EV Charging Stations Data analysis

Team Members: Sandro Juric, Scotty Seethoff

### AI usage disclaimer
I have used the Copilot in this project, primarily for code refactoring in VS Code

# Business Understanding

Driving pure electric vehicles comes with new challenges that new and existing consumers face. Of those range anxiety is the most significant hurdle that EV driver faces in most countries. A common issue with most electric vehicles is that they typically need to be recharged after around two hundred miles of driving. While many EV drivers have home charging stations, the problem apperent during road trips. 

Therefore, having charging stations available becomes paramount to a successful road trip experience in an EV car. There are four main factors to consider when dealing with EV charging stations. Firstly, are there any charging stations available in the area where you will be driving? Secondly, do those available stations support fast DC charging and which rate can you expect the output to be? Next, do the available charging station support the type of charging connector you can have? Finally, are the stations available in good working conditions, or mostly offline?

The dataset analysed is for the global EV charging stations and will describe the state of each country EV infrastructure. Real-time data from 123 active countries, spanning 2010 to 2026, has been gathered to illustrate the growth of electric vehicles and assess the reliability of individual stations. Along with geographic data, the data includes station status, maximum power output, available connectors, and which year it was created.

My hypothesis is that the global EV infrastructure has grown significantly over the years and that will continue to grow in the decades to come. With the analysis we will explore if that is the case and what countries have reached a critical density of reliable EV station for an everyday consumer to feel comfortable driving anywhere in their EV without extensive planning.

Dataset: https://www.kaggle.com/datasets/sohails07/global-ev-charging-station-network-2010-2026

Questions seeking to answers:

1. Has the EV charging infrastructure rollout increase over time year over year?
2. Which countries have enough EV chargers?

# Data Understanding

In [1]:
import pandas as pd
import numpy as np

# Suppress SettingWithCopyWarning
pd.options.mode.chained_assignment = None  # default='warn'
# Ensure future behavior for downcasting is explicit
pd.set_option('future.no_silent_downcasting', True)

df = pd.read_csv('global_ev_charging_station.csv')

# select relevant columns
dfSelect = df[['StationID', 'Country', 'Town', 'Latitude', 'Longitude', 'MaxPowerKW', 'ConnectionTypes', 'StatusType', 'YearCreated']]

# remap connection types to broader categories
connection_type_map = {
    'CHAdeMO': 'Fast',
    'CCS (Type 1)': 'Fast',
    'CCS (Type 2)': 'Fast',
    'Tesla Supercharger': 'Fast',
    'Type 2': 'Standard',
    'Type 1': 'Standard',
    'J1772': 'Standard',
    'GB/T': 'Fast',
    'Schuko': 'Slow',
    'Ceeform (Blue)': 'Slow',
    'Ceeform (Red)': 'Slow'
}




dfSelect['ConnectionSpeed'] = dfSelect['ConnectionTypes'].map(connection_type_map).fillna('Unknown')

# remap status types to simplified categories
status_type_map = { 
    'Operational': True,
    'Under Construction': False,
    'Planned': False,
    'Temporarily Unavailable': False,
    'Removed': False
}

# convert StatusType to boolean
dfSelect['StatusType'] = dfSelect['StatusType'].map(status_type_map).astype('bool', errors='ignore')

# handle missing values for Town and Country
dfSelect['Town'] = dfSelect['Town'].fillna('Unknown')
dfSelect['Country'] = dfSelect['Country'].fillna('Unknown')

# rename columns for clarity
dfSelect.rename(columns={
    'Town': 'City', 
    'MaxPowerKW': 'Rate', 
    'YearCreated': 'Built', 
    'ConnectionSpeed': 'Charge Rate', 
    'StatusType': 'Is Online', 
    'Latitude': 'Lat', 
    'Longitude': 'Long', 
    'StationID':'Station ID'}, inplace=True)

# format and display of the first few rows
# used the pandas Styler for better visualization in Jupyter Notebooks https://pandas.pydata.org/docs/user_guide/style.html
display(dfSelect.head(10).style.background_gradient(axis=None, cmap="YlGnBu"))

,Station ID,Country,City,Lat,Long,Rate,ConnectionTypes,Is Online,Built,Charge Rate
0,473101,United Kingdom,Unknown,51.371709,-0.116955,250.000000,CCS (Type 2),True,2026,Fast
1,472908,United Kingdom,Wythenshawe,53.366453,-2.269799,11.000000,Type 2 (Socket Only),True,2026,Unknown
2,472220,United Kingdom,Belfast,54.586836,-5.926869,50.000000,CCS (Type 2),True,2025,Fast
3,471954,United Kingdom,Dungannon,54.509453,-6.768057,22.000000,Type 2 (Socket Only),True,2025,Unknown
4,471953,United Kingdom,Armagh,54.350644,-6.641787,22.000000,Type 2 (Socket Only),True,2025,Unknown
5,471952,United Kingdom,Armagh,54.349971,-6.642684,22.000000,Type 2 (Socket Only),True,2025,Unknown
6,471951,United Kingdom,Craigavon,54.435689,-6.411357,22.000000,Type 2 (Socket Only),True,2025,Unknown
7,471950,United Kingdom,Portadown,54.433089,-6.410223,22.000000,Type 2 (Socket Only),True,2025,Unknown
8,471949,United Kingdom,Lurgan,54.456608,-6.338349,22.000000,Type 2 (Socket Only),True,2025,Unknown
9,471948,United Kingdom,Rathfriland,54.236068,-6.162224,22.000000,Type 2 (Socket Only),True,2025,Unknown


### Data Cleanup
Picked the relavent features for analysis and discarded rest. Mapped over Charge Rate based on common known mapping and set unknown data to "Other". Mapped over the operational status to boolean, it's either online or not.  Renamed columns for easer read and formated the output.

In [2]:
dfSelect.info()


print("\n")
print("Data Summary Statistics:")

# Display summary statistics
display(dfSelect.describe(include='all'))

# Display number of records with unknown values with percentages
unknown_cities_count = dfSelect[dfSelect['City'] == 'Unknown'].shape[0]
print(f"Number of records with unknown cities: {unknown_cities_count} ({unknown_cities_count / len(dfSelect) * 100:.2f}%)")

unknown_countries_count = dfSelect[dfSelect['Country'] == 'Unknown'].shape[0]
print(f"Number of records with unknown countries: {unknown_countries_count} ({unknown_countries_count / len(dfSelect) * 100:.2f}%)")

unknown_connection_types_count = dfSelect[dfSelect['Charge Rate'] == 'Unknown'].shape[0]
print(f"Number of records with 'Unknown' connection types: {unknown_connection_types_count} ({unknown_connection_types_count / len(dfSelect) * 100:.2f}%)")

offline_stations_count = dfSelect[dfSelect['Is Online'] == False].shape[0]
print(f"Number of offline stations: {offline_stations_count} ({offline_stations_count / len(dfSelect) * 100:.2f}%)")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257585 entries, 0 to 257584
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Station ID       257585 non-null  int64  
 1   Country          257585 non-null  object 
 2   City             257585 non-null  object 
 3   Lat              257585 non-null  float64
 4   Long             257585 non-null  float64
 5   Rate             257585 non-null  float64
 6   ConnectionTypes  252709 non-null  object 
 7   Is Online        257585 non-null  bool   
 8   Built            257585 non-null  int64  
 9   Charge Rate      257585 non-null  object 
dtypes: bool(1), float64(3), int64(2), object(4)
memory usage: 17.9+ MB


Data Summary Statistics:


,Station ID,Country,City,Lat,Long,Rate,ConnectionTypes,Is Online,Built,Charge Rate
count,257585.000000,257585,257585,257585.000000,257585.000000,257585.000000,252709,257585,257585.000000,257585
unique,NaN,123,43777,NaN,NaN,NaN,474,2,NaN,2
top,NaN,United States,Unknown,NaN,NaN,NaN,Type 1 (J1772),True,NaN,Unknown
freq,NaN,86904,22756,NaN,NaN,NaN,75403,252412,NaN,240001
mean,214027.344244,NaN,NaN,43.024583,-32.331054,36.094556,NaN,NaN,2021.177984,NaN
std,112354.931522,NaN,NaN,13.060504,57.567388,1971.171374,NaN,NaN,3.404048,NaN
min,2389.000000,NaN,NaN,-55.811599,-164.848855,0.000000,NaN,NaN,2010.000000,NaN
25%,127474.000000,NaN,NaN,38.785981,-81.528641,3.700000,NaN,NaN,2019.000000,NaN
50%,213775.000000,NaN,NaN,44.293968,-3.117260,11.000000,NaN,NaN,2023.000000,NaN
75%,289084.000000,NaN,NaN,51.343085,7.690003,22.000000,NaN,NaN,2024.000000,NaN


Number of records with unknown cities: 22756 (8.83%)
Number of records with unknown countries: 0 (0.00%)
Number of records with 'Unknown' connection types: 240001 (93.17%)
Number of offline stations: 5173 (2.01%)


## Data Analysis

Unknown city in data is below 10% and we still have long & lat data for those records so these can be safely ignored as mapping it can still be done to a map.

There are no unknown contries and the number of offline station is statistically insignificant to matter for the final analysis so we can ignore those.

However connection types are a big issue as the majority of the records didn't fall with the mapping range, we'll need to adjust that before proceding.

In [4]:
# get unique connection types before mapping
unique_connection_types = dfSelect['ConnectionTypes'].unique()

# print unique connection types before mapping
print('Total number of unique connection types:', len(unique_connection_types))

print("Sample unique connection types:")
print(unique_connection_types[:10])

print("\n")
print("Issue seems that most records have multiple connection types listed, causing them to be categorized as 'Unknown' after mapping.")

# remapping logic could be improved by splitting multiple connection types and categorizing based on priority.
def categorize_connection_type(connection_types):
    if pd.isna(connection_types):
        return 'Unknown'
    
    fast_types = ['CCS', 'CHADEMO', 'SUPERCHARGER', 'NACS', 'GB-T DC', 'GB/T 20234.3', 'CHAOJI', 'NACS / TESLA SUPERCHARGER']
    standard_types = ['TYPE 2', 'TYPE 1', 'J1772', 'GB-T AC', 'GB/T 20234.2', 'TESLA', 'TYPE I', 'TYPE II']
    slow_types = ['SCHUKO', 'CEE 7', 'BS1363','BS1363 3 PIN 13 AMP', 'COMMANDO', 'CEE 3', 'CEE 5', 'EUROPLUG', 
                  'NEMA', 'SCAME', 'IEC 60309', 'CEE+', 'T13', 'TYPE M', '3 PIN', 'BLUE COMMANDO', 'CEE 5 PIN', 'CEE 3 PIN',
                  'NEMA 5-20R', 'NEMA 14-50', 'NEMA 5-15R', 'NEMA 14-50, NEMA 5-15R', 'NEMA 14-30', 'NEMA 6-15', 'NEMA 5-20R, ,NEMA TT-30R',
                  'NEMA 6-20', 'EUROPLUG 2-PIN', 'THREE PHASE 5-PIN', 'GB-T AC - GB/T 20234.2', 'CEE 7/5', 'IEC 60309 3-PIN', 'IEC 60309 5-PIN',
                  'CEE 7/4 - SCHUKO - TYPE F', 'SCAME TYPE 3A', 'SCAME TYPE 3C', 'SCAME TYPE 3G', 'SP INDUCTIVE' ,'AVCON CONNECTOR', 'LP INDUCTIVE',
                  'GB-T DC - GB/T 20234.3', 'CEE+ 7 PIN', 'WIRELESS CHARGING', 'T13 - SEC1011']

    # Split the connection types and clean them, strip characters in parentheses
    types = [ctype.strip().upper().split('(')[0].strip() for ctype in connection_types.split(',')]

    if any(t in fast_types for t in types):
        return 'Fast'
    if any(t in standard_types for t in types):
        return 'Standard'  
    if any(t in slow_types for t in types):
        return 'Slow'
        
    return 'Unknown'

dfSelect['Charge Rate'] = dfSelect['ConnectionTypes'].apply(categorize_connection_type)

print("\nAfter improved mapping:")
unknown_connection_types_count = dfSelect[dfSelect['Charge Rate'] == 'Unknown'].shape[0]
print(f"Number of records with 'Unknown' connection types: {unknown_connection_types_count} ({unknown_connection_types_count / len(dfSelect) * 100:.2f}%)")

unique_connection_types = dfSelect['ConnectionTypes'].where(dfSelect['Charge Rate'] == 'Unknown').unique()
print('Total number of unique connection types after improved mapping:', len(unique_connection_types))

print("Sample unique connection types after improved mapping:")
display(dfSelect.head(10).style.background_gradient(axis=None, cmap="YlGnBu"))

Total number of unique connection types: 475
Sample unique connection types:
['CCS (Type 2)' 'Type 2 (Socket Only)'
 'CCS (Type 2), Type 2 (Socket Only)' 'Type 2 (Tethered Connector)' nan
 'CCS (Type 2), CHAdeMO' 'CCS (Type 1)'
 'CCS (Type 2), CHAdeMO, Type 2 (Tethered Connector)' 'CHAdeMO'
 'CCS (Type 2), CHAdeMO, Type 2 (Socket Only)']


Issue seems that most records have multiple connection types listed, causing them to be categorized as 'Unknown' after mapping.

After improved mapping:
Number of records with 'Unknown' connection types: 4876 (1.89%)
Total number of unique connection types after improved mapping: 1
Sample unique connection types after improved mapping:


,Station ID,Country,City,Lat,Long,Rate,ConnectionTypes,Is Online,Built,Charge Rate
0,473101,United Kingdom,Unknown,51.371709,-0.116955,250.000000,CCS (Type 2),True,2026,Fast
1,472908,United Kingdom,Wythenshawe,53.366453,-2.269799,11.000000,Type 2 (Socket Only),True,2026,Standard
2,472220,United Kingdom,Belfast,54.586836,-5.926869,50.000000,CCS (Type 2),True,2025,Fast
3,471954,United Kingdom,Dungannon,54.509453,-6.768057,22.000000,Type 2 (Socket Only),True,2025,Standard
4,471953,United Kingdom,Armagh,54.350644,-6.641787,22.000000,Type 2 (Socket Only),True,2025,Standard
5,471952,United Kingdom,Armagh,54.349971,-6.642684,22.000000,Type 2 (Socket Only),True,2025,Standard
6,471951,United Kingdom,Craigavon,54.435689,-6.411357,22.000000,Type 2 (Socket Only),True,2025,Standard
7,471950,United Kingdom,Portadown,54.433089,-6.410223,22.000000,Type 2 (Socket Only),True,2025,Standard
8,471949,United Kingdom,Lurgan,54.456608,-6.338349,22.000000,Type 2 (Socket Only),True,2025,Standard
9,471948,United Kingdom,Rathfriland,54.236068,-6.162224,22.000000,Type 2 (Socket Only),True,2025,Standard


This impoved mapping fixed the issue with charging rate attribute and now we got less then 2% of records with unknown values.

# Data Visualization

# Dimensionality Reduction Method